In [1]:
import os
import numpy as np
import glob
import PIL.Image as Image
from tqdm import tqdm
from Dataloader import *
from torch.utils.data import ConcatDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split

from torchvision import models
from torchsummary import summary
import torch.optim as optim
from time import time
from IPython.display import clear_output
from Loss import *
from torch.utils.tensorboard.writer import SummaryWriter
from HyperParameterSearch import *

We check that we're running on a GPU

In [2]:
if torch.cuda.is_available():
    print("This code will run on GPU.")
else:
    print("The code will run on CPU.")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

This code will run on GPU.


Taking a look at some of the images from the two datasets

In [3]:
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ]
)

drive_dataset = DRIVE(train=True, transform=transform)
drive_train_size = int(0.8 * len(drive_dataset))
drive_val_size = len(drive_dataset) - drive_train_size
drive_train, drive_val = random_split(drive_dataset, [drive_train_size, drive_val_size])
drive_test = DRIVE(train=False, transform=transform)

ph2_dataset = PH2(train=True, transform=transform)
ph2_train_size = int(0.8 * len(ph2_dataset))
ph2_val_size = len(ph2_dataset) - ph2_train_size
ph2_train, ph2_val = random_split(ph2_dataset, [ph2_train_size, ph2_val_size])
ph2_test = PH2(train=False, transform=transform)                    



# Designing the model architecture

Simple encoder decoder network class

In [4]:

class EncDec(nn.Module):
    def __init__(self):
        super().__init__()

        # encoder (downsampling)
        self.enc_conv0 = nn.Conv2d(3, 64, 3, padding=1)
        self.pool0 = nn.MaxPool2d(2, 2)  # 128 -> 64
        self.enc_conv1 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)  # 64 -> 32
        self.enc_conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)  # 32 -> 16
        self.enc_conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)  # 16 -> 8

        # bottleneck
        self.bottleneck_conv = nn.Conv2d(64, 64, 3, padding=1)

        # decoder (upsampling)
        self.upsample0 = nn.Upsample(32)  # 8 -> 16
        self.dec_conv0 = nn.Conv2d(64, 64, 3, padding=1)
        self.upsample1 = nn.Upsample(64)  # 16 -> 32
        self.dec_conv1 = nn.Conv2d(64, 64, 3, padding=1)
        self.upsample2 = nn.Upsample(128)  # 32 -> 64
        self.dec_conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.upsample3 = nn.Upsample(256)  # 64 -> 128
        self.dec_conv3 = nn.Conv2d(64, 1, 3, padding=1)

    def forward(self, x):
        # encoder
        e0 = self.pool0(F.relu(self.enc_conv0(x)))
        e1 = self.pool1(F.relu(self.enc_conv1(e0)))
        e2 = self.pool2(F.relu(self.enc_conv2(e1)))
        e3 = self.pool3(F.relu(self.enc_conv3(e2)))

        # bottleneck
        b = F.relu(self.bottleneck_conv(e3))

        # decoder
        d0 = F.relu(self.dec_conv0(self.upsample0(b)))
        d1 = F.relu(self.dec_conv1(self.upsample1(d0)))
        d2 = F.relu(self.dec_conv2(self.upsample2(d1)))
        d3 = self.dec_conv3(self.upsample3(d2))  # no activation
        return d3
    

# Actually training model

In [5]:
model = EncDec().to(device)
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
         MaxPool2d-2         [-1, 64, 128, 128]               0
            Conv2d-3         [-1, 64, 128, 128]          36,928
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]          36,928
         MaxPool2d-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          36,928
         MaxPool2d-8           [-1, 64, 16, 16]               0
            Conv2d-9           [-1, 64, 16, 16]          36,928
         Upsample-10           [-1, 64, 32, 32]               0
           Conv2d-11           [-1, 64, 32, 32]          36,928
         Upsample-12           [-1, 64, 64, 64]               0
           Conv2d-13           [-1, 64, 64, 64]          36,928
         Upsample-14         [-1, 64, 1

Specifying hyperparameters

In [6]:
print("Current working directory:", os.getcwd())

run_dir = "HPSearch"
os.makedirs(run_dir, exist_ok=True)

# Define the loss function
loss_function = bce_loss
results = {}

hyperparameters = {
    "number of classes": 2,
    "device": device,
    "image size": (256, 256),
    "backbone": "SimpleEncDec",
    "torch home": "TorchvisionModels",
    "network name": "Test-0",
    "beta1": 0.9, # Default values for Adam
    "beta2": 0.999, # Default values for Adam
    "epsilon": 1e-08, # Default values for Adam
    "number of workers": 0, 
    "momentum": 0.9,
    "weight decay": 0.0005,
    "scheduler": "Yes",
}


hyperparameter_grid = {
    'batch size': [1, 2, 4],
    'step size': [5, 3, 2],
    'learning rate': [1e-3, 1e-4, 1e-5],
    "epochs": [5, 10, 20],
    'gamma': [0.8, 0.9, 0.7],
    'momentum': [0.9, 0.95],
    'optimizer': ['Adam', 'sgd'], 
    
}

hyperparameter_grid = {
    'batch size': [1, 2],
    'step size': [5],
    'learning rate': [1e-3],
    "epochs": [5],
    'gamma': [0.8],
    'momentum': [0.9],
    'optimizer': ['Adam', 'sgd'], 
    
}

Current working directory: /zhome/c8/c/169006/Repos/ImageSegmentation


Creating dataset loaders

In [7]:
from torch.utils.tensorboard.writer import SummaryWriter

trainset = ConcatDataset([drive_train, ph2_train])
# train_loader = torch.utils.data.DataLoader(trainset, batch_size=hyperparameters['batch size'], shuffle=True)

valset = ConcatDataset([drive_val, ph2_val])
# val_loader = torch.utils.data.DataLoader(valset, batch_size=hyperparameters['batch size'], shuffle=False)

testset = ConcatDataset([drive_test, ph2_test])
# test_loader = torch.utils.data.DataLoader(testset, batch_size=hyperparameters['batch size'], shuffle=False)

print(f"Created a new Dataset for training of length: {len(trainset)}")
print(f"Created a new Dataset for validation of length: {len(valset)}")
print(f"Created a new Dataset for testing of length: {len(testset)}")

Created a new Dataset for training of length: 48
Created a new Dataset for validation of length: 12
Created a new Dataset for testing of length: 30


Conducting Hyperparameter Search

In [8]:
# Perform hyperparameter search
samples = create_combinations(hyperparameter_grid)
# samples = sample_hyperparameters(hyperparameter_grid, 15)

print(f"Number of combinations: {len(samples)} (amount of models to test)\n\n")
best_hp = hyperparameter_search(model, hyperparameters["backbone"], loss_function, device, trainset,
                                valset, testset, samples, hyperparameters, run_dir)
results[hyperparameters["backbone"]] = best_hp
print(f"Best hyperparameters for {hyperparameters['backbone']}: {best_hp}")

print(f"\n\nResults: {results}")

Number of combinations: 4 (amount of models to test)


Current hyper parameters: {'batch size': 1, 'step size': 5, 'learning rate': 0.001, 'epochs': 5, 'gamma': 0.8, 'momentum': 0.9, 'optimizer': 'Adam'}
Created a new Dataloader for training with batch size: 1
Created a new Dataloader for validation with batch size: 1
Created a new Dataloader for testing with batch size: 1


Training Epoch [1/5]:   0%|                                                                                   …

Validation Epoch [1/5]:   0%|                                                                                 …

Current learning rate: [0.001]


Training Epoch [2/5]:   0%|                                                                                   …

...

Validation Epoch [4/5]:   0%|                                                                                 …

Current learning rate: [0.001]


Training Epoch [5/5]:   0%|                                                                                   …

Validation Epoch [5/5]:   0%|                                                                                 …

Current learning rate: [0.0008]
Got 1292412/1572864 with accuracy 82.169%




############### Finished hyperparameter search! ###############
Best hyperparameters for SimpleEncDec: {'batch size': 1, 'step size': 5, 'learning rate': 0.001, 'epochs': 5, 'gamma': 0.8, 'momentum': 0.9, 'optimizer': 'Adam', 'number of classes': 2, 'device': device(type='cuda'), 'image size': (256, 256), 'backbone': 'SimpleEncDec', 'torch home': 'TorchvisionModels', 'network name': 'Test-0', 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08, 'number of workers': 0, 'weight decay': 0.0005, 'scheduler': 'Yes'}


Results: {'SimpleEncDec': {'batch size': 1, 'step size': 5, 'learning rate': 0.001, 'epochs': 5, 'gamma': 0.8, 'momentum': 0.9, 'optimizer': 'Adam', 'number of classes': 2, 'device': device(type='cuda'), 'image size': (256, 256), 'backbone': 'SimpleEncDec', 'torch home': 'TorchvisionModels', 'network name': 'Test-0', 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08, 'number of workers': 0, 'weight decay': 0

In [9]:
def predict(model, data):
    model.eval()  # testing mode
    Y_pred = [F.sigmoid(model(X_batch.to(device))) for X_batch, _ in data]
    return np.array(Y_pred)

# Need implementation of evaluation metrics